# 네이버 영화 리뷰 분석

In [1]:
import re
import joblib
import numpy as np 
import pandas as pd

In [3]:
train_df = pd.read_csv('../../../Machine-Learning/00.data/NaverMovie/ratings_train.txt', sep='\t')
test_df = pd.read_csv('../../../Machine-Learning/00.data/NaverMovie/ratings_test.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [4]:
train_df.shape, test_df.shape


((150000, 3), (50000, 3))

In [5]:
# 중복 여부 확인
train_df['document'].nunique()

146182

In [6]:
# 중복 샘플 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [7]:
# Null값 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [8]:
# Null값 제거
train_df = train_df.dropna(how = 'any')
train_df.shape

(146182, 3)

In [9]:
# 긍정, 부정 레이블 값의 갯수
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋에도 적용

In [10]:
# 중복 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [11]:
# Null 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

In [12]:
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [14]:
test_df.to_csv('../static/data/naver/movie_test0.tsv', sep='\t', index=False)

### 텍스트 전처리

In [15]:
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
train_df['document'].replace('', np.nan, inplace=True)
train_df.isnull().sum()

id            0
document    391
label         0
dtype: int64

In [17]:
train_df = train_df.dropna(how = 'any')
train_df.shape

(145791, 3)

In [18]:
train_df.to_csv('../static/data/naver/movie_train.tsv', sep='\t', index=False)

- 테스트 데이터셋

In [19]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [20]:
indices = list(test_df['document'] != '')
indices[:3]

[True, False, True]

In [21]:
df = pd.read_csv('../static/data/naver/movie_test0.tsv', sep='\t')
new_df = df.loc[indices,:]
new_df.shape

(48995, 3)

In [22]:
new_df.to_csv('../static/data/naver/movie_test.tsv', sep='\t', index=False)

In [23]:
test_df = pd.read_csv('../static/data/naver/movie_test.tsv', sep='\t')
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df.shape

(48995, 3)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [25]:
from konlpy.tag import Okt
okt = Okt()

In [26]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [27]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df['document']):
    morphs = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = ' '.join([word for word in morphs if not word in stopwords]) # 불용어 제거
    X_train.append(temp_X)

In [28]:
X_test = []
for sentence in tqdm_notebook(test_df['document']):
    morphs = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = ' '.join([word for word in morphs if not word in stopwords]) # 불용어 제거
    X_test.append(temp_X)

In [29]:
y_train = train_df.label.values
y_test = test_df.label.values

In [30]:
X_train[:3]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다']

In [31]:
y_train[:3]

array([0, 1, 0], dtype=int64)

### Case 1. CountVectorizer + LogisticRegression

In [32]:
pipeline = Pipeline([
    ('count_vect', CountVectorizer()),
    ('lr_clf', LogisticRegression())
])
params = ({
    'count_vect__ngram_range': [(1,1), (1,2)],
    'count_vect__max_df': [300, 700],
    'lr_clf__C': [1, 10]
})

In [33]:
%time pipeline.fit(X_train, y_train)

Wall time: 3.3 s


Pipeline(steps=[('count_vect', CountVectorizer()),
                ('lr_clf', LogisticRegression())])

In [34]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  2.3min finished
{'count_vect__max_df': 700, 'count_vect__ngram_range': (1, 2), 'lr_clf__C': 1} 0.8180957672284297


In [35]:
best_count_lr = grid_pipe.best_estimator_
pred_count_lr = best_count_lr.predict(X_test)
accuracy_score(y_test, pred_count_lr)

0.8196958873354424

In [36]:
joblib.dump(best_count_lr, '../static/model/naver_count_lr.pkl')

['../static/model/naver_count_lr.pkl']

### Case 2. CountVectorizer + NaiveBayes

In [37]:
pipeline = Pipeline([
    ('count_vect', CountVectorizer()),
    ('nb_clf', MultinomialNB())
])
params = ({
    'count_vect__ngram_range': [(1,1), (1,2)],
    'count_vect__max_df': [300, 700]
})

In [38]:
%time pipeline.fit(X_train, y_train)

Wall time: 1.38 s


Pipeline(steps=[('count_vect', CountVectorizer()), ('nb_clf', MultinomialNB())])

In [39]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   30.5s finished
{'count_vect__max_df': 700, 'count_vect__ngram_range': (1, 2)} 0.8266491072837144


In [40]:
best_count_nb = grid_pipe.best_estimator_
pred_count_nb = best_count_nb.predict(X_test)
accuracy_score(y_test, pred_count_nb)

0.8284722930911318

In [41]:
joblib.dump(best_count_nb, '../static/model/naver_count_nb.pkl')

['../static/model/naver_count_nb.pkl']

### Case 3. TfidfVectorizer + LogisticRegression

In [42]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer()),
    ('lr_clf', LogisticRegression())
])
params = ({
    'tfidf_vect__ngram_range': [(1,1), (1,2)],
    'tfidf_vect__max_df': [300, 700],
    'lr_clf__C': [1, 10]
})

In [43]:
%time pipeline.fit(X_train, y_train)

Wall time: 3.09 s


Pipeline(steps=[('tfidf_vect', TfidfVectorizer()),
                ('lr_clf', LogisticRegression())])

In [44]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  2.1min finished
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.8204072953748859


In [45]:
best_tfidf_lr = grid_pipe.best_estimator_
pred_tfidf_lr = best_tfidf_lr.predict(X_test)
accuracy_score(y_test, pred_tfidf_lr)

0.8203490152056332

In [46]:
joblib.dump(best_tfidf_lr, '../static/model/naver_tfidf_lr.pkl')

['../static/model/naver_tfidf_lr.pkl']

### Case 4. TfidfVectorizer + NaiveBayes

In [47]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer()),
    ('nb_clf', MultinomialNB())
])
params = ({
    'tfidf_vect__ngram_range': [(1,1), (1,2)],
    'tfidf_vect__max_df': [300, 700]
})

In [48]:
%time pipeline.fit(X_train, y_train)

Wall time: 1.36 s


Pipeline(steps=[('tfidf_vect', TfidfVectorizer()), ('nb_clf', MultinomialNB())])

In [49]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3,
                         scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   33.0s finished
{'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.8286108195979175


In [50]:
best_tfidf_nb = grid_pipe.best_estimator_
pred_tfidf_nb = best_tfidf_nb.predict(X_test)
accuracy_score(y_test, pred_tfidf_nb)

0.8298806000612308

In [51]:
joblib.dump(best_tfidf_nb, '../static/model/naver_tfidf_nb.pkl')

['../static/model/naver_tfidf_nb.pkl']

### TEST
 - TEST data set

In [52]:
index = 100
review = test_df.document[index]
label = test_df.label[index]
review, label

('이렇게 지겨울수가', 0)

In [53]:
test_data = []
morphs = okt.morphs(review, stem=True) # 토큰화
temp_X = ' '.join([word for word in morphs if not word in stopwords]) # 불용어 제거
test_data.append(temp_X)
test_data


['이렇게 지겹다']

In [54]:
pred_cl = best_count_lr.predict(test_data)
pred_cn = best_count_nb.predict(test_data)
pred_tl = best_tfidf_lr.predict(test_data)
pred_tn = best_tfidf_nb.predict(test_data)

In [55]:
label, pred_cl[0], pred_cn[0], pred_tl[0], pred_tn[0]


(0, 0, 0, 0, 0)

In [56]:
review = '이런 사랑영화가 다시 나올 수 있을까?'

In [57]:
review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", review)
test_data = []
morphs = okt.morphs(review, stem=True) # 토큰화
temp_X = ' '.join([word for word in morphs if not word in stopwords]) # 불용어 제거
test_data.append(temp_X)
test_data

['이렇다 사랑 영화 다시 나오다 수 있다']

In [58]:
pred_cl = best_count_lr.predict(test_data)
pred_cn = best_count_nb.predict(test_data)
pred_tl = best_tfidf_lr.predict(test_data)
pred_tn = best_tfidf_nb.predict(test_data)

In [59]:
pred_cl[0], pred_cn[0], pred_tl[0], pred_tn[0]

(1, 1, 1, 1)